In [1]:
import numpy as np
import pandas as pd

# The data used for training should be strictly in this format
# -root directory
# -data
# --order_products__prior.csv
# --order_products__train.csv
# --orders.csv
# --products.csv
# -code...

prior_address = "./data/order_products__prior.csv"
train_address = "./data/order_products__train.csv"
orders_address = "./data/orders.csv"
products_address = "./data/products.csv"

print("-------Well, the data proccsing costs lots of time so I decide to print something-------")
print("-------Not only to make it not so boring, but also let you know our little program is still working-------")
print("-------This artificial idiot is going to read files from your disk-------")
print("-------But plz give it some time as the files are a little bit large-------")
prior = pd.read_csv(prior_address)
train = pd.read_csv(train_address)
orders = pd.read_csv(orders_address)
products = pd.read_csv(products_address)
print("-------load file completed-------")

# update 1, memory cost is one of the major problems in lightgbm
# so we are going to transfer the default type from int64 to np.uint16 or np.uint32
# based on data_analysis.ipynb we learned each colunm's maximum value and the maximum value 
# of usigned int8, unsigned int16 and unsigned int32
print("-------convert type start-------")

products["product_id"] = products["product_id"].astype(np.uint16)
products["aisle_id"] = products["aisle_id"].astype(np.uint8)
products["department_id"] = products["department_id"].astype(np.uint8)
products = products.drop(['product_name'],axis = 1)
products.info()
# in products we noticed that the "products name" has the same function with product_id
# and id is much more simple to handle, so we are going to remove "products name" column

prior["order_id"] = prior["order_id"].astype(np.uint32)
prior["product_id"] = prior["product_id"].astype(np.uint16)
prior["add_to_cart_order"] = prior["add_to_cart_order"].astype(np.uint8)
prior["reordered"] = prior["reordered"].astype(np.uint8)

train["order_id"] = train["order_id"].astype(np.uint32)
train["product_id"] = train["product_id"].astype(np.uint16)
train["add_to_cart_order"] = train["add_to_cart_order"].astype(np.uint8)
train["reordered"] = train["reordered"].astype(np.uint8)

orders["order_id"] = orders["order_id"].astype(np.uint32)
orders["user_id"] = orders["user_id"].astype(np.uint32)
orders["order_number"] = orders["order_number"].astype(np.uint8)
orders["order_dow"] = orders["order_dow"].astype(np.uint8)
orders["order_hour_of_day"] = orders["order_hour_of_day"].astype(np.uint8)

print("-------convert type finished-------")
print("-------Compute extra feature start-------")

# For each specific product, calculate how many times it has been bought
# if the RAM cost is too high, go adjust the variable type here
products_tmp = pd.DataFrame()
products_tmp['product_ordered_num'] = prior.groupby(prior.product_id).size().astype(np.uint32)
products_tmp['product_reordered_num'] = prior['reordered'].groupby(prior.product_id).sum().astype(np.float32)
products_tmp['product_reordered_rate'] = (products_tmp.product_reordered_num / products_tmp.product_ordered_num).astype(np.float32)

products = products.join(products_tmp, on='product_id')
#products.set_index('product_id', drop=False, inplace=True)
del products_tmp

orders.set_index('order_id', inplace=True, drop=False)
prior = prior.join(orders, on='order_id', rsuffix='tmp')
#prior.info()
prior.drop(["order_idtmp"],axis = 1,inplace = True)

user_tmp = pd.DataFrame()
user_tmp['orders_frequency_days'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
user_tmp['total_orders'] = orders.groupby('user_id').size().astype(np.int16)
#user_tmp.info()
#print(user_tmp.head(5))
user_tmp['item_sum'] = prior.groupby('user_id').size().astype(np.int16)
user_tmp['products_list'] = prior.groupby('user_id')['product_id'].apply(set)
user_tmp['unique_products_num'] = prior.groupby('user_id')['product_id'].nunique()

user_tmp['item_sum'] = prior.groupby('user_id').size().astype(np.int16)
user_tmp['items_order'] = (user_tmp.item_sum / user_tmp.total_orders).astype(np.float32)

#print(user_tmp.head(5))
#print(products.head(5))
#print(orders.head(5))
#print(prior.info())
print("finished")

-------Well, the data proccsing costs lots of time so I decide to print something-------
-------Not only to make it not so boring, but also let you know our little program is still working-------
-------This artificial idiot is going to read files from your disk-------
-------But plz give it some time as the files are a little bit large-------
-------load file completed-------
-------convert type start-------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 3 columns):
product_id       49688 non-null uint16
aisle_id         49688 non-null uint8
department_id    49688 non-null uint8
dtypes: uint16(1), uint8(2)
memory usage: 194.2 KB
-------convert type finished-------
-------Compute extra feature start-------
finished


In [2]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 6 columns):
product_id                49688 non-null uint16
aisle_id                  49688 non-null uint8
department_id             49688 non-null uint8
product_ordered_num       49677 non-null float64
product_reordered_num     49677 non-null float32
product_reordered_rate    49677 non-null float32
dtypes: float32(2), float64(1), uint16(1), uint8(2)
memory usage: 970.5 KB


In [3]:
#user_tmp.info()
#print(user_tmp.head(1))
user_tmp.info()
print(user_tmp.products_list[1])

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 206209 entries, 1 to 206209
Data columns (total 6 columns):
orders_frequency_days    206209 non-null float32
total_orders             206209 non-null int16
item_sum                 206209 non-null int16
products_list            206209 non-null object
unique_products_num      206209 non-null int64
items_order              206209 non-null float32
dtypes: float32(2), int16(2), int64(1), object(1)
memory usage: 7.1+ MB
{17122, 196, 26405, 46149, 14084, 13032, 26088, 39657, 12427, 25133, 35951, 38928, 10258, 30450, 49235, 10326, 13176, 41787}


In [4]:
products["product_ordered_num"] = products["product_ordered_num"].astype(np.float32)
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 6 columns):
product_id                49688 non-null uint16
aisle_id                  49688 non-null uint8
department_id             49688 non-null uint8
product_ordered_num       49677 non-null float32
product_reordered_num     49677 non-null float32
product_reordered_rate    49677 non-null float32
dtypes: float32(3), uint16(1), uint8(2)
memory usage: 776.5 KB


In [5]:
orders.info()
print(orders.head(5))

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 3421083 entries, 2539329 to 272231
Data columns (total 7 columns):
order_id                  uint32
user_id                   uint32
eval_set                  object
order_number              uint8
order_dow                 uint8
order_hour_of_day         uint8
days_since_prior_order    float64
dtypes: float64(1), object(1), uint32(2), uint8(3)
memory usage: 114.2+ MB
          order_id  user_id eval_set  order_number  order_dow  \
order_id                                                        
2539329    2539329        1    prior             1          2   
2398795    2398795        1    prior             2          3   
473747      473747        1    prior             3          3   
2254736    2254736        1    prior             4          4   
431534      431534        1    prior             5          4   

          order_hour_of_day  days_since_prior_order  
order_id                                             
2539329       

In [6]:
orders_tmp = prior.copy()

In [7]:
print(orders_tmp.head(5))
orders_tmp.info()

   order_id  product_id  add_to_cart_order  reordered  user_id eval_set  \
0         2       33120                  1          1   202279    prior   
1         2       28985                  2          1   202279    prior   
2         2        9327                  3          0   202279    prior   
3         2       45918                  4          1   202279    prior   
4         2       30035                  5          0   202279    prior   

   order_number  order_dow  order_hour_of_day  days_since_prior_order  
0             3          5                  9                     8.0  
1             3          5                  9                     8.0  
2             3          5                  9                     8.0  
3             3          5                  9                     8.0  
4             3          5                  9                     8.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
or

In [8]:
orders_tmp1 = pd.DataFrame()
orders_tmp1["userproduct_order_num"] = orders_tmp.groupby(["user_id","product_id"]).size()
orders_tmp1["user_last_order"] = orders_tmp.groupby(["user_id","product_id"])["order_id"].agg('max')
print(orders_tmp1.head(5))

                    userproduct_order_num  user_last_order
user_id product_id                                        
1       196                            10          3367565
        10258                           9          3367565
        10326                           1           431534
        12427                          10          3367565
        13032                           3          2550362


In [9]:
orders_tmp = pd.merge(orders_tmp,orders_tmp1,how = "inner",on = ["user_id","product_id"])
orders_tmp.info()
print(orders_tmp.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 12 columns):
order_id                  uint32
product_id                uint16
add_to_cart_order         uint8
reordered                 uint8
user_id                   uint32
eval_set                  object
order_number              uint8
order_dow                 uint8
order_hour_of_day         uint8
days_since_prior_order    float64
userproduct_order_num     int64
user_last_order           uint32
dtypes: float64(1), int64(1), object(1), uint16(1), uint32(3), uint8(5)
memory usage: 1.5+ GB
   order_id  product_id  add_to_cart_order  reordered  user_id eval_set  \
0         2       33120                  1          1   202279    prior   
1    104690       33120                  2          1   202279    prior   
2    132412       33120                  1          1   202279    prior   
3   2808715       33120                  1          1   202279    prior   
4   2894949       33120  

In [10]:
print(orders_tmp[(orders_tmp.user_id == 1)&(orders_tmp.product_id == 196)])
print(orders_tmp[(orders_tmp.user_id == 1)&(orders_tmp.product_id == 10258)])
print(orders_tmp[(orders_tmp.user_id == 1)&(orders_tmp.product_id == 10326)])
print(orders_tmp[(orders_tmp.user_id == 1)&(orders_tmp.product_id == 12427)])
print(orders_tmp[(orders_tmp.user_id == 1)&(orders_tmp.product_id == 13032)])

          order_id  product_id  add_to_cart_order  reordered  user_id  \
15038552    431534         196                  1          1        1   
15038553    473747         196                  1          1        1   
15038554    550135         196                  1          1        1   
15038555   2254736         196                  1          1        1   
15038556   2295261         196                  4          1        1   
15038557   2398795         196                  1          1        1   
15038558   2539329         196                  1          0        1   
15038559   2550362         196                  1          1        1   
15038560   3108588         196                  2          1        1   
15038561   3367565         196                  1          1        1   

         eval_set  order_number  order_dow  order_hour_of_day  \
15038552    prior             5          4                 15   
15038553    prior             3          3                 12   
1

In [11]:
print(orders_tmp[(orders_tmp.order_id == 550135)])

          order_id  product_id  add_to_cart_order  reordered  user_id  \
15038554    550135         196                  1          1        1   
15038564    550135       12427                  3          1        1   
15038574    550135       10258                  2          1        1   
15038583    550135       25133                  4          1        1   
17012815    550135       13032                  5          1        1   

         eval_set  order_number  order_dow  order_hour_of_day  \
15038554    prior             7          1                  9   
15038564    prior             7          1                  9   
15038574    prior             7          1                  9   
15038583    prior             7          1                  9   
17012815    prior             7          1                  9   

          days_since_prior_order  userproduct_order_num  user_last_order  
15038554                    20.0                     10          3367565  
15038564            

In [12]:
print(orders_tmp[(orders_tmp.product_id == 196)])

          order_id  product_id  add_to_cart_order  reordered  user_id  \
1093            13         196                  4          0    45082   
4373            68         196                  5          1    96594   
4374        685669         196                  1          1    96594   
4375        800858         196                  5          1    96594   
4376       1446782         196                  1          1    96594   
4377       2183136         196                  1          1    96594   
4378       2633548         196                  1          0    96594   
6734           100         196                  2          0    62781   
6735        601311         196                  1          1    62781   
6736       3386141         196                  1          1    62781   
13944          200         196                  2          1    21425   
13945       463002         196                  1          1    21425   
13946       511821         196                  1  

In [13]:
orders_tmp["userproduct_order_num"] = orders_tmp["userproduct_order_num"].astype(np.uint16)
orders_tmp["days_since_prior_order"] = orders_tmp["days_since_prior_order"].astype(np.float32)
orders_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 12 columns):
order_id                  uint32
product_id                uint16
add_to_cart_order         uint8
reordered                 uint8
user_id                   uint32
eval_set                  object
order_number              uint8
order_dow                 uint8
order_hour_of_day         uint8
days_since_prior_order    float32
userproduct_order_num     uint16
user_last_order           uint32
dtypes: float32(1), object(1), uint16(2), uint32(3), uint8(5)
memory usage: 1.2+ GB


In [14]:
del orders_tmp1

In [15]:
train_set = orders[orders.eval_set =='train']
train_set["days_since_prior_order"] = train_set["days_since_prior_order"].astype(np.float32)
train_set.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 131209 entries, 1187899 to 272231
Data columns (total 7 columns):
order_id                  131209 non-null uint32
user_id                   131209 non-null uint32
eval_set                  131209 non-null object
order_number              131209 non-null uint8
order_dow                 131209 non-null uint8
order_hour_of_day         131209 non-null uint8
days_since_prior_order    131209 non-null float32
dtypes: float32(1), object(1), uint32(2), uint8(3)
memory usage: 3.9+ MB


/Users/yuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
order_id_list = []
product_id_list = []
label = []
train.set_index(["order_id","product_id"],inplace = True, drop = False)
train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1384617 entries, (1, 49302) to (3421070, 4724)
Data columns (total 4 columns):
order_id             1384617 non-null uint32
product_id           1384617 non-null uint16
add_to_cart_order    1384617 non-null uint8
reordered            1384617 non-null uint8
dtypes: uint16(1), uint32(1), uint8(2)
memory usage: 22.4 MB


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1384617 entries, (1, 49302) to (3421070, 4724)
Data columns (total 4 columns):
order_id             1384617 non-null uint32
product_id           1384617 non-null uint16
add_to_cart_order    1384617 non-null uint8
reordered            1384617 non-null uint8
dtypes: uint16(1), uint32(1), uint8(2)
memory usage: 22.4 MB


In [18]:
i = 0
for data in train_set.itertuples():
    i+=1
    if(i%10000 == 0):
        print(i)
    product_tmp_list = []
    order_id = data.order_id
    user_id = data.user_id
    for i in user_tmp.products_list[user_id]:
        product_id_list.append(i)
        order_id_list.append(order_id)
        product_tmp_list.append(i)
    if(len(order_id_list) != len(product_id_list)):
        print(len(order_id_list))
        print(len(product_id_list))
    label += [(order_id,product) in train.index for product in product_tmp_list]

In [19]:
train_dict = {'order_id': order_id_list,'product_id':product_id_list}
print(len(order_id_list))
print(len(product_id_list))

8474661
8474661


In [20]:
train_set = pd.DataFrame(train_dict)
train_set["user_id"] = train_set.order_id.map(orders.user_id)
train_set['user_orders_num'] = train_set.user_id.map(user_tmp.total_orders)
train_set['user_history_item_num'] = train_set.user_id.map(user_tmp.item_sum)
train_set['user_history_different_item_num'] = train_set.user_id.map(user_tmp.unique_products_num)
train_set['user_order_frequency_bydays'] = train_set.user_id.map(user_tmp.orders_frequency_days)
train_set['user_average_order'] = train_set.user_id.map(user_tmp.items_order)
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8474661 entries, 0 to 8474660
Data columns (total 8 columns):
order_id                           int64
product_id                         int64
user_id                            uint32
user_orders_num                    int16
user_history_item_num              int16
user_history_different_item_num    int64
user_order_frequency_bydays        float32
user_average_order                 float32
dtypes: float32(2), int16(2), int64(3), uint32(1)
memory usage: 323.3 MB


In [21]:
train_set["order_hour_of_day"] =  train_set.order_id.map(orders.order_hour_of_day)
train_set["days_since_prior_order"] = train_set.order_id.map(orders.days_since_prior_order)
train_set["day_weight"] = train_set.days_since_prior_order/train_set.user_order_frequency_bydays
train_set["aisle_id"] = train_set.product_id.map(products.aisle_id)
train_set["department_id"] = train_set.product_id.map(products.department_id)
train_set["product_ordered_num"] = train_set.product_id.map(products.product_ordered_num)
train_set["product_reordered_num"] = train_set.product_id.map(products.product_reordered_num)
train_set["product_reordered_rate"] = train_set.product_id.map(products.product_reordered_rate)
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8474661 entries, 0 to 8474660
Data columns (total 16 columns):
order_id                           int64
product_id                         int64
user_id                            uint32
user_orders_num                    int16
user_history_item_num              int16
user_history_different_item_num    int64
user_order_frequency_bydays        float32
user_average_order                 float32
order_hour_of_day                  uint8
days_since_prior_order             float64
day_weight                         float64
aisle_id                           float64
department_id                      float64
product_ordered_num                float32
product_reordered_num              float32
product_reordered_rate             float32
dtypes: float32(5), float64(4), int16(2), int64(3), uint32(1), uint8(1)
memory usage: 687.0 MB


In [22]:
orders_tmp["sub_index"] = orders_tmp.user_id*1000000+orders_tmp.product_id
orders_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 13 columns):
order_id                  uint32
product_id                uint16
add_to_cart_order         uint8
reordered                 uint8
user_id                   uint32
eval_set                  object
order_number              uint8
order_dow                 uint8
order_hour_of_day         uint8
days_since_prior_order    float32
userproduct_order_num     uint16
user_last_order           uint32
sub_index                 uint32
dtypes: float32(1), object(1), uint16(2), uint32(4), uint8(5)
memory usage: 1.4+ GB


In [23]:
train_set["sub_index"] = train_set.user_id*1000000+train_set.product_id
train_set["userproduct_order_num"] = train_set.sub_index.map(orders_tmp.userproduct_order_num)
train_set["userproduct_order_rate"] = train_set.userproduct_order_num/train_set.user_orders_num
train_set["userproduct_last_order"] = train_set.sub_index.map(orders_tmp.user_last_order)
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8474661 entries, 0 to 8474660
Data columns (total 20 columns):
order_id                           int64
product_id                         int64
user_id                            uint32
user_orders_num                    int16
user_history_item_num              int16
user_history_different_item_num    int64
user_order_frequency_bydays        float32
user_average_order                 float32
order_hour_of_day                  uint8
days_since_prior_order             float64
day_weight                         float64
aisle_id                           float64
department_id                      float64
product_ordered_num                float32
product_reordered_num              float32
product_reordered_rate             float32
sub_index                          int64
userproduct_order_num              float64
userproduct_order_rate             float64
userproduct_last_order             float64
dtypes: float32(5), float64(7), int16(2), 

In [24]:
train_set["userproduct_reorder_rate"] = train_set.product_ordered_num/train_set.user_orders_num

In [25]:
train_set.drop(["sub_index"],axis = 1, inplace = True)

In [26]:
train_set.to_csv("./data/train_set.csv")

In [27]:
print(label[1])

True


In [28]:
label_dict = {"label":label}
label_df = pd.DataFrame(label_dict)
label_df.to_csv("./data/label.csv")

In [29]:
train_set = orders[orders.eval_set =='test']
train_set["days_since_prior_order"] = train_set["days_since_prior_order"].astype(np.float32)
train_set.info()
order_id_list = []
product_id_list = []
train.set_index(["order_id","product_id"],inplace = True, drop = False)
i = 0
for data in train_set.itertuples():
    i+=1
    if(i%10000 == 0):
        print(i)
    product_tmp_list = []
    order_id = data.order_id
    user_id = data.user_id
    for i in user_tmp.products_list[user_id]:
        product_id_list.append(i)
        order_id_list.append(order_id)
        product_tmp_list.append(i)
    if(len(order_id_list) != len(product_id_list)):
        print(len(order_id_list))
        print(len(product_id_list))
train_dict = {'order_id': order_id_list,'product_id':product_id_list}
train_set = pd.DataFrame(train_dict)
train_set["user_id"] = train_set.order_id.map(orders.user_id)
train_set['user_orders_num'] = train_set.user_id.map(user_tmp.total_orders)
train_set['user_history_item_num'] = train_set.user_id.map(user_tmp.item_sum)
train_set['user_history_different_item_num'] = train_set.user_id.map(user_tmp.unique_products_num)
train_set['user_order_frequency_bydays'] = train_set.user_id.map(user_tmp.orders_frequency_days)
train_set['user_average_order'] = train_set.user_id.map(user_tmp.items_order)
train_set["order_hour_of_day"] =  train_set.order_id.map(orders.order_hour_of_day)
train_set["days_since_prior_order"] = train_set.order_id.map(orders.days_since_prior_order)
train_set["day_weight"] = train_set.days_since_prior_order/train_set.user_order_frequency_bydays
train_set["aisle_id"] = train_set.product_id.map(products.aisle_id)
train_set["department_id"] = train_set.product_id.map(products.department_id)
train_set["product_ordered_num"] = train_set.product_id.map(products.product_ordered_num)
train_set["product_reordered_num"] = train_set.product_id.map(products.product_reordered_num)
train_set["product_reordered_rate"] = train_set.product_id.map(products.product_reordered_rate)
orders_tmp["sub_index"] = orders_tmp.user_id*1000000+orders_tmp.product_id
train_set["sub_index"] = train_set.user_id*1000000+train_set.product_id
train_set["userproduct_order_num"] = train_set.sub_index.map(orders_tmp.userproduct_order_num)
train_set["userproduct_order_rate"] = train_set.userproduct_order_num/train_set.user_orders_num
train_set["userproduct_last_order"] = train_set.sub_index.map(orders_tmp.user_last_order)
train_set["userproduct_reorder_rate"] = train_set.product_ordered_num/train_set.user_orders_num
train_set.drop(["sub_index"],axis = 1, inplace = True)
train_set.to_csv("./data/test_set.csv")

/Users/yuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'pandas.core.frame.DataFrame'>
UInt64Index: 75000 entries, 2774568 to 803273
Data columns (total 7 columns):
order_id                  75000 non-null uint32
user_id                   75000 non-null uint32
eval_set                  75000 non-null object
order_number              75000 non-null uint8
order_dow                 75000 non-null uint8
order_hour_of_day         75000 non-null uint8
days_since_prior_order    75000 non-null float32
dtypes: float32(1), object(1), uint32(2), uint8(3)
memory usage: 2.2+ MB
